**INSTITUTO DE EDUCAÇÃO SUPERIOR – IESB**

**PÓS-GRADUAÇÃO EM CIÊNCIA DE DADOS**

Data Mining e Machine Learning II

Carolina Silva Baptistella

Matrícula:1931133089


**Introdução**

No presente estudo foi utilizada a base de dados HME_Data (Home Equity)disponibilizada no Kaggle. É uma base de dados de empréstimos onde encontramos 5960 amostras distribuidas em 13 variáveis. 
O objetivo é analisar o comportamento distribuídos entre os bons e maus pagadores.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**Dicionário de Variáveis**

* BAD - cliente inadimplente no empréstimo 0 = empréstimo reembolsado
* LOAN - Montante da solicitação de empréstimo
* MORTDUE - Valor devido da hipoteca existente
* VALUE - Valor da propriedade atual
* REASON - DebtCon = consolidação da dívida / HomeImp - melhoramento da casa
* JOB - Seis categorias profissionais
* YOJ - Anos no emprego atual
* DEROG - Número de principais relatórios depreciativos
* DELINQ - Número de linhas de crédito inadimplentes
* CLAGE - Idade da linha comercial mais antiga em meses
* NINQ - Número de linhas de crédito recentes
* CLNO - Número de linhas de crédito
* DEBTINC - Rácio dívida / rendimento


**Importação dos Dados**

In [ ]:
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df2 = df.copy()

In [ ]:
display(df.sample(10).T)
display(df.shape)

**Tratamento dos Dados**

Antes do início das análises a base de dados foi tratada, pois foram identificados linhas que não possuiam informações e estas foram eliminadas.

In [ ]:
df.info()

**Análise Exploratória**

Em seguida fazemos uma análise apresentando um resumo geral dos dados utilizados como média, desvio padrão, entre outros.

In [ ]:
df.describe().T

In [ ]:
def missing_values(data_frame):
    total = data_frame.isnull().count()
    missing = data_frame.isnull().sum()
    missing_percent = missing/total * 100
    display(missing_percent)
    
missing_values(df)



**Análise**

A análise inicial, mostra possui um número pequeno de 'maus pagadores'(=1).


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df.hist(figsize=(20,10))

**Verificando os histogramas**

In [ ]:
numeric_feats = [c for c in df.columns if df[c].dtype != 'object' and c not in ['BAD']]
df_numeric_feats = df[numeric_feats]

In [ ]:
df_numeric_feats.hist(figsize=(20,8), bins=30)
plt.tight_layout() 

Separados por tipo de trabalhos, pode-se verificar que a maior parte foi classificado como outros.


In [ ]:
jobs = df['JOB'].dropna().unique()
plt.figure(figsize=(14,15))
c=1
for i in jobs:
    plt.subplot(7,1,c)
    plt.title(i)
    df[df['JOB'] == i]['VALUE'].hist(bins=20)
    c+=1
plt.tight_layout() 

Verificado a média de tempo de trabalho por tipo de trabalho e valor médio

In [ ]:
df.groupby('JOB')['YOJ'].mean()

In [ ]:
value_mean_by_job = df.groupby('JOB')['VALUE'].mean()
value_mean_by_job

Verificada a quantidade de bons e maus pagadores de acordo com o tipo de trabalho

In [ ]:
qtd1 = df.groupby(['BAD'])['JOB'].value_counts()

qtd1

Verificamos que a média de valores dos empréstimos está na faixa de 10000 a 30000

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
fig=sns.boxplot(x='LOAN', data=df, orient='v')
fig.set_title('BoxPlot de Empréstimo (LOAN)')
fig.set_ylabel('Valores de Empréstimos')

**Criação da base para modelo Random Forest**

Criação de um dataframe para este modelo

In [ ]:
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scikitplot as skplt
from sklearn.model_selection import cross_val_score

In [ ]:
df_rf = pd.get_dummies(df, df.dtypes[(df.dtypes==np.object) | (df.dtypes=='category')].index.values, drop_first=True)
df_rf.head()

In [ ]:
train, test = train_test_split(df_rf, test_size=0.20, random_state=42)

In [ ]:
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
df_rf.info()

**Modelo Random Forest Classifier**

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, n_estimators = 150, random_state=150)

In [ ]:
feats = [c for c in df_rf.columns if c not in ['BAD']]
feats

In [ ]:
df_rf['BAD'] = pd.to_numeric(df['BAD'])

In [ ]:
missing_values(df)

In [ ]:
rf.fit(train[feats],train['BAD'])

**Prevendo os dados de validação**

In [ ]:
preds = rf.predict(valid[feats])

**Acurácia do modelo**

In [ ]:
accuracy_score(valid['BAD'], preds_val)

**Modelo Preditivo**

In [ ]:
preds_test = rf.predict(test[feats])

preds_test

In [ ]:
skplt.metrics.plot_confusion_matrix(test['BAD'], preds_test)

Podemos verificar uma acurácia acima de 90% onde concluímos que o modelo é muito bom.